In [2]:
import os
from azure.mgmt.cosmosdb import CosmosDBManagementClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv()

subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
print(f"Using subscription ID: {subscription_id}")

credential = DefaultAzureCredential()

client = CosmosDBManagementClient(credential=credential, subscription_id=subscription_id)

print(f"Using Cosmos DB client: {client}")

Using subscription ID: 8e0b36ef-c452-4f0c-b4ee-c6b28721721c
Using Cosmos DB client: <azure.mgmt.cosmosdb._cosmos_db_management_client.CosmosDBManagementClient object at 0x0000027541F630B0>


In [5]:
# props = container_client.read()
props = client.read()
pk_paths = props.get('partitionKey', {}).get('paths', [])
print("Partition key paths:", pk_paths)
field_name = pk_paths[0].lstrip('/') if pk_paths else None
print("Partition key field name:", field_name)

AttributeError: 'CosmosDBManagementClient' object has no attribute 'read'

In [3]:
from azure.cosmos import PartitionKey, exceptions as cosmos_exceptions

# Use discovered PK field
pk_field = field_name or 'partitionKey'  # fallback if not found
# partition_key_value = "Contoso"
partition_key_value = "claim_id"
# Parameterized query to avoid injection/quoting issues
query = f"SELECT * FROM c WHERE c.{pk_field} = @pk"
parameters = [{"name": "@pk", "value": partition_key_value}]

try:
    # Prefer single-partition query when value is known
    items = list(container_client.query_items(
        query=query,
        parameters=parameters,
        partition_key=partition_key_value,
        enable_cross_partition_query=False
    ))
except cosmos_exceptions.CosmosHttpResponseError as e:
    # Fallback if the item isn’t in that logical partition or PK mismatch (e.g., wrong field/value)
    print(f"Single-partition query failed: {e}. Retrying cross-partition…")
    items = list(container_client.query_items(
        query=query,
        parameters=parameters,
        enable_cross_partition_query=True
    ))

print(f"Returned {len(items)} item(s)")
for item in items:
    print(item)

NameError: name 'field_name' is not defined

In [ ]:
# Composite partition key variant: [tenantId, accountId]
# This cell assumes pk_paths like ['/tenantId', '/accountId']
from azure.cosmos import exceptions as cosmos_exceptions

# Ensure we have both PK fields
pk_fields = [p.lstrip('/') for p in pk_paths]
needs_composite = len(pk_fields) > 1
print("Composite PK required:", needs_composite, "->", pk_fields)

# Provide values (edit as needed)
tenant_value = "Contoso"
account_value = "Acc003"  # set to a specific account id if known

# If account_value not provided, discover one for this tenant (small sample)
if needs_composite and not account_value:
    sample_query = f"SELECT TOP 1 c.{pk_fields[1]} AS acct FROM c WHERE c.{pk_fields[0]} = @tenant"
    sample_params = [{"name": "@tenant", "value": tenant_value}]
    try:
        sample = list(container_client.query_items(
            query=sample_query,
            parameters=sample_params,
            enable_cross_partition_query=True
        ))
        if sample:
            account_value = sample[0].get('acct')
            print("Discovered account_value:", account_value)
    except cosmos_exceptions.CosmosHttpResponseError as e:
        print("Discovery query failed:", e)

# Build composite PK and run query
if needs_composite and account_value is not None:
    comp_pk = [tenant_value, account_value]
    query = f"SELECT * FROM c WHERE c.{pk_fields[0]} = @t AND c.{pk_fields[1]} = @a"
    parameters = [
        {"name": "@t", "value": tenant_value},
        {"name": "@a", "value": account_value},
    ]
    try:
        items = list(container_client.query_items(
            query=query,
            parameters=parameters,
            partition_key=comp_pk,
            enable_cross_partition_query=False
        ))
    except cosmos_exceptions.CosmosHttpResponseError as e:
        print(f"Composite single-partition query failed: {e}. Retrying cross-partition…")
        items = list(container_client.query_items(
            query=query,
            parameters=parameters,
            enable_cross_partition_query=True
        ))
    print(f"Returned {len(items)} item(s) with composite PK")
    for item in items[:10]:
        print(item)
else:
    print("Composite PK not required or account_value unresolved. Set account_value to proceed.")

In [17]:
# Display values for a specific accountId
from azure.cosmos import exceptions as cosmos_exceptions

# Set the account id you want to view
account_id = "Acc003"  # change as needed

# Detect composite PK and resolve tenantId for this account (if present)
pk_fields = [p.lstrip('/') for p in pk_paths]
needs_composite = len(pk_fields) > 1 and 'tenantId' in pk_fields and 'accountId' in pk_fields

tenant_value = None
if needs_composite:
    try:
        sample = list(container_client.query_items(
            query="SELECT TOP 1 c.tenantId AS t FROM c WHERE c.accountId = @aid",
            parameters=[{"name": "@aid", "value": account_id}],
            enable_cross_partition_query=True
        ))
        if sample:
            tenant_value = sample[0].get('t')
            print("Resolved tenantId:", tenant_value)
    except cosmos_exceptions.CosmosHttpResponseError as e:
        print("Tenant discovery failed:", e)

items = []
if needs_composite and tenant_value:
    # Use composite partition key for efficient single-partition query
    comp_pk = [tenant_value, account_id]
    q = "SELECT * FROM c WHERE c.tenantId = @t AND c.accountId = @a"
    params = [
        {"name": "@t", "value": tenant_value},
        {"name": "@a", "value": account_id},
    ]
    try:
        items = list(container_client.query_items(
            query=q,
            parameters=params,
            partition_key=comp_pk,
            enable_cross_partition_query=False
        ))
    except cosmos_exceptions.CosmosHttpResponseError as e:
        print("Composite single-partition query failed:", e, "Falling back to cross-partition.")
        items = list(container_client.query_items(
            query=q,
            parameters=params,
            enable_cross_partition_query=True
        ))
else:
    # Generic cross-partition query if we can't resolve tenant for this account
    q = "SELECT * FROM c WHERE c.accountId = @a"
    params = [{"name": "@a", "value": account_id}]
    items = list(container_client.query_items(
        query=q,
        parameters=params,
        enable_cross_partition_query=True
    ))

print(f"Found {len(items)} item(s) for accountId={account_id}")
for i, item in enumerate(items[:20], 1):
    print(f"{i:02d}. id={item.get('id')} tenantId={item.get('tenantId')} accountId={item.get('accountId')}")
# Uncomment to print full documents
# import json
# print(json.dumps(items, indent=2))

Resolved tenantId: Contoso
Found 6 item(s) for accountId=Acc003
01. id=Acc003 tenantId=Contoso accountId=Acc003
02. id=Transaction0007 tenantId=Contoso accountId=Acc003
03. id=Transaction0008 tenantId=Contoso accountId=Acc003
04. id=Transaction0009 tenantId=Contoso accountId=Acc003
05. id=Acc003-10 tenantId=Contoso accountId=Acc003
06. id=Acc003-311 tenantId=Contoso accountId=Acc003


In [18]:
# Render JSON payload for a given accountId as professional HTML
from IPython.display import HTML, display
import json
from datetime import datetime

# Reuse account_id from previous cell or set here
account_id = account_id if 'account_id' in globals() else 'Acc003'

# Ensure we have the items for this account; query if not present
if 'items' not in globals() or not items:
    # Resolve tenant
    sample = list(container_client.query_items(
        query="SELECT TOP 1 c.tenantId AS t FROM c WHERE c.accountId = @aid",
        parameters=[{"name": "@aid", "value": account_id}],
        enable_cross_partition_query=True
    ))
    tenant_value = sample[0].get('t') if sample else None
    if tenant_value:
        comp_pk = [tenant_value, account_id]
        q = "SELECT * FROM c WHERE c.tenantId = @t AND c.accountId = @a"
        params = [
            {"name": "@t", "value": tenant_value},
            {"name": "@a", "value": account_id},
        ]
        try:
            items = list(container_client.query_items(
                query=q,
                parameters=params,
                partition_key=comp_pk,
                enable_cross_partition_query=False
            ))
        except Exception:
            items = list(container_client.query_items(
                query=q,
                parameters=params,
                enable_cross_partition_query=True
            ))
    else:
        items = list(container_client.query_items(
            query="SELECT * FROM c WHERE c.accountId = @a",
            parameters=[{"name": "@a", "value": account_id}],
            enable_cross_partition_query=True
        ))

pretty_docs = [json.dumps(doc, indent=2, ensure_ascii=False) for doc in items]

# Simple, clean styling; Bootstrap-like card layout
style = """
<style>
  .container{font-family:Segoe UI,Roboto,Helvetica,Arial,sans-serif;max-width:1100px;margin:20px auto;padding:0 12px;}
  .header{display:flex;justify-content:space-between;align-items:center;margin-bottom:16px}
  .title{font-size:22px;font-weight:600;color:#0f172a}
  .meta{font-size:12px;color:#64748b}
  .grid{display:grid;grid-template-columns:repeat(auto-fill,minmax(340px,1fr));gap:12px}
  .card{border:1px solid #e5e7eb;border-radius:8px;box-shadow:0 1px 2px rgba(0,0,0,.04);background:#fff;overflow:hidden}
  .card .card-header{background:#f8fafc;border-bottom:1px solid #e5e7eb;padding:8px 12px;font-weight:600;color:#334155}
  .card pre{margin:0;padding:12px;background:#ffffff;color:#0f172a;white-space:pre-wrap;word-wrap:break-word}
  .badge{display:inline-block;background:#eef2ff;color:#3730a3;border:1px solid #c7d2fe;border-radius:999px;padding:3px 8px;font-size:12px}
</style>
"""

generated_at = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%SZ')
header = f"""
<div class="container">
  <div class="header">
    <div class="title">Account payload</div>
    <div class="meta">accountId: <span class="badge">{account_id}</span> • docs: <span class="badge">{len(pretty_docs)}</span> • generated: {generated_at} UTC</div>
  </div>
  <div class="grid">
"""

cards = []
for idx, doc in enumerate(pretty_docs, start=1):
    doc_id = items[idx-1].get('id', f'doc-{idx}')
    cards.append(f"""
    <div class="card">
      <div class="card-header">Document {idx} • id: {doc_id}</div>
      <pre>{doc}</pre>
    </div>
    """)

footer = """
  </div>
</div>
"""

html = style + header + "\n".join(cards) + footer
display(HTML(html))

C:\Users\andrehe\AppData\Local\Temp\ipykernel_30224\2008313990.py:62: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  generated_at = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%SZ')


## Connect to msagthack-cosmos-jlhj53wabtsro Azure Cosmos DB

In [7]:
# Create Cosmos data-plane client with DefaultAzureCredential and connect to `insurance_claims` container
from azure.identity import DefaultAzureCredential
from azure.cosmos import CosmosClient, exceptions as cosmos_exceptions

account_name = "msagthack-cosmos-jlhj53wabtsro"  # provided
endpoint = f"https://{account_name}.documents.azure.com:443/"
credential = DefaultAzureCredential()

# Create data-plane client
dp_client = CosmosClient(endpoint, credential=credential)
print("Cosmos client created for:", endpoint)

# Find the database that has a container named 'insurance_claims'
container_name = "crash_reports"
matched_db = None
try:
    for db in dp_client.list_databases():
        db_client = dp_client.get_database_client(db['id'])
        for container_props in db_client.list_containers():
            if container_props['id'] == container_name:
                matched_db = db['id']
                break
        if matched_db:
            break
    if not matched_db:
        raise RuntimeError(f"Container '{container_name}' not found in any database. Check the name.")

    database_client = dp_client.get_database_client(matched_db)
    container_client = database_client.get_container_client(container_name)
    # Quick smoke test: read container properties
    props = container_client.read()
    print(f"Connected to database='{matched_db}', container='{container_name}'. PK paths: {props.get('partitionKey', {}).get('paths', [])}")
except cosmos_exceptions.CosmosHttpResponseError as e:
    print("Cosmos HTTP error:", e)
except Exception as ex:
    print("Setup error:", ex)

Cosmos client created for: https://msagthack-cosmos-jlhj53wabtsro.documents.azure.com:443/
Connected to database='insurance_claims', container='crash_reports'. PK paths: ['/claim_id']
Connected to database='insurance_claims', container='crash_reports'. PK paths: ['/claim_id']


In [8]:
# Display all items (sampled) across dp_client databases/containers in professional HTML
from IPython.display import HTML, display
import json
from datetime import datetime, timezone

if 'dp_client' not in globals():
    raise RuntimeError("dp_client not defined. Run the Cosmos client setup cell first.")

# Config
max_items_per_container = 50  # to avoid huge output; increase if needed

collected = []  # (db_id, container_id, doc)
for db in dp_client.list_databases():
    db_id = db['id']
    db_client = dp_client.get_database_client(db_id)
    for container_props in db_client.list_containers():
        c_id = container_props['id']
        c_client = db_client.get_container_client(c_id)
        try:
            # Prefer a simple query to sample items; ORDER BY _ts desc if available
            query = "SELECT * FROM c"
            docs = []
            for doc in c_client.query_items(query=query, enable_cross_partition_query=True):
                docs.append(doc)
                if len(docs) >= max_items_per_container:
                    break
            for d in docs:
                collected.append((db_id, c_id, d))
        except Exception as e:
            collected.append((db_id, c_id, {"_error": str(e)}))

# Build HTML
style = """
<style>
  .container{font-family:Segoe UI,Roboto,Helvetica,Arial,sans-serif;max-width:1200px;margin:20px auto;padding:0 12px;}
  .header{display:flex;justify-content:space-between;align-items:center;margin-bottom:16px}
  .title{font-size:22px;font-weight:600;color:#0f172a}
  .meta{font-size:12px;color:#64748b}
  .grid{display:grid;grid-template-columns:repeat(auto-fill,minmax(360px,1fr));gap:12px}
  .card{border:1px solid #e5e7eb;border-radius:8px;box-shadow:0 1px 2px rgba(0,0,0,.04);background:#fff;overflow:hidden}
  .card .card-header{background:#f8fafc;border-bottom:1px solid #e5e7eb;padding:8px 12px;font-weight:600;color:#334155}
  .card pre{margin:0;padding:12px;background:#ffffff;color:#0f172a;white-space:pre-wrap;word-wrap:break-word}
  .badge{display:inline-block;background:#eef2ff;color:#3730a3;border:1px solid #c7d2fe;border-radius:999px;padding:3px 8px;font-size:12px}
</style>
"""

generated_at = datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%SZ')
header = f"""
<div class=container>
  <div class=header>
    <div class=title>All items (sample)</div>
    <div class=meta>containers scanned: <span class=badge>{len({(d,c) for d,c,_ in collected})}</span> • total docs: <span class=badge>{len(collected)}</span> • generated: {generated_at}</div>
  </div>
  <div class=grid>
"""

cards = []
for idx, (db_id, c_id, doc) in enumerate(collected, start=1):
    doc_id = doc.get('id', f'doc-{idx}') if isinstance(doc, dict) else f'doc-{idx}'
    body = json.dumps(doc, indent=2, ensure_ascii=False)
    cards.append(f"""
    <div class=card>
      <div class=card-header>#{idx} • db: {db_id} • container: {c_id} • id: {doc_id}</div>
      <pre>{body}</pre>
    </div>
    """)

footer = """
  </div>
</div>
"""

html = style + header + "\n".join(cards) + footer
display(HTML(html))

print(f"Rendered {len(collected)} item(s) across {len(set((d,c) for d,c,_ in collected))} container(s). Set max_items_per_container to control volume.")

Rendered 4 item(s) across 1 container(s). Set max_items_per_container to control volume.
